<a href="https://colab.research.google.com/github/JBenedictM/501-A4-TF/blob/master/501_A4_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/My Drive/heart.csv"
train_path = "/content/drive/My Drive/heart_train.csv"
test_path = "/content/drive/My Drive/heart_test.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create Train and Test csv files
Splits Heart.csv into 80% training data and 20% testing data

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys

data = pd.read_csv(data_path)
y_chd = data.chd
X = data.drop("chd", axis=1)
X = X.drop("row.names", axis=1)
#data = data.drop("", axis=1)
#print(X)

x_train, x_test, y_train, y_test = train_test_split(data, y_chd, test_size=0.2)

#print(x_train.shape)
#print(x_train)

#print(x_test.shape)
#print(x_test)

x_train.to_csv("/content/drive/My Drive/heart_train.csv", encoding="utf-8", sep='\t', index=False)
x_test.to_csv("/content/drive/My Drive/heart_test.csv", encoding="utf-8", sep='\t', index=False)

# Load and Process Data

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pandas as pd
import sys

PREDICT_COLUMN = "chd"
PREDICT_LABELs = [0, 1]

# load csv data
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        data_path, 
        label_name=PREDICT_COLUMN,
        batch_size=1,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    
    return dataset

def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label


def normalize_numeric_data(data, mean, std):
    return (data-mean)/std



In [0]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

      
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features["numeric"] = numeric_features

        return features, labels


In [0]:
# instaniate training and testing data

NAME_COLUMN = "row.names"
SELECT_COLUMNS =  ["sbp", "tobacco", "ldl", "adiposity", "famhist", "typea", "obesity", "alcohol", "age"] + [PREDICT_COLUMN]
NUMERIC_COLUMNS = ["sbp", "tobacco", "ldl", "adiposity", "typea", "obesity", "alcohol", "age"]
SELECT_COLUMNS_NUM = NUMERIC_COLUMNS + [PREDICT_COLUMN]
DEFAULT_NUM = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

raw_train_data = get_dataset(train_path, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_path, select_columns=SELECT_COLUMNS)

#show_batch(raw_train_data)
#print()
#show_batch(raw_test_data)

# pack numeric data types
#temp_train_numeric = get_dataset(train_path, select_columns=SELECT_COLUMNS_NUM, column_defaults=DEFAULT_NUM)
#show_batch(temp_train_numeric)
#print(len(list(raw_test_data)))

example_batch, labels_batch = next(iter(temp_train_numeric))

#packed_dataset = temp_train_numeric.map(pack)

#for features, label in packed_dataset.take(1):
 #   print(features.numpy())
  #  print()
   # print(label.numpy())

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_COLUMNS))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_COLUMNS))
show_batch(packed_train_data)

example_batch, labels_batch = next(iter(packed_train_data))

# normalize data
# load data and calculate mean and standard deviation of training data
#desc = pd.read_csv(train_path, names=SELECT_COLUMNS, index_col=SELECT_COLUMNS, usecols=NUMERIC_COLUMNS, sep='\t')
desc = pd.read_csv(train_path, sep='\t')[NUMERIC_COLUMNS].describe()
print(desc)
MEAN = np.array(desc.T["mean"])
STD = np.array(desc.T["std"])

#sys.exit(1)

normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column("numeric", normalizer_fn=normalizer, shape=[len(NUMERIC_COLUMNS)])
numeric_columns = [numeric_column]
#print("Before")
#print(example_batch["numeric"])

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
#print("After")
#print(numeric_layer(example_batch).numpy())

# enumerate categorical columns
CATEGORIES = {
    "famhist" : ["Absent", "Present"]
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(  
              key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

#categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
#print(categorical_layer(example_batch).numpy())
#sys.exit(0)

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)
#print(preprocessing_layer(example_batch).numpy())
#print(labels_batch)

train_data = packed_train_data.shuffle(500)
#train_data = preprocessing_layer(.numpy())
test_data = packed_test_data
        

famhist             : [b'Absent']
numeric             : [[1.660e+02 7.000e-02 4.030e+00 2.929e+01 5.300e+01 2.837e+01 0.000e+00
  2.700e+01]]
              sbp     tobacco         ldl  ...     obesity     alcohol         age
count  369.000000  369.000000  369.000000  ...  369.000000  369.000000  369.000000
mean   138.818428    3.796504    4.785474  ...   26.041274   17.706423   43.555556
std     21.265713    4.758530    2.099181  ...    4.096126   25.539654   14.591085
min    101.000000    0.000000    0.980000  ...   14.700000    0.000000   15.000000
25%    124.000000    0.100000    3.280000  ...   23.110000    0.510000   32.000000
50%    134.000000    2.200000    4.370000  ...   25.870000    7.710000   45.000000
75%    150.000000    5.600000    5.810000  ...   28.110000   23.970000   57.000000
max    218.000000   31.200000   15.330000  ...   46.580000  147.190000   64.000000

[8 rows x 8 columns]


# Create and Train Model

# Base Model

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)
print("--Fit model--")
val_loss_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=20, )
checkpoint_val = tf.keras.callbacks.ModelCheckpoint("base_heart_model.h5", monitor="val_loss", mode="min", save_best_only=True, save_weights_only=True, verbose=1)
#checkpoint_train = tf.keras.callbacks.ModelCheckpoint("base_heart_model.h5", monitor="loss", mode="min", save_best_only=True, save_weights_only=True, verbose=1)

cb = [val_loss_callback, checkpoint_val]

model.fit(train_data, epochs=99999999999, verbose=1, validation_data=(test_data), callbacks=cb)
#tf.saved_model.save(model, "best_heart_model.h5")
#loaded_model = tf.keras.models.load_model("best_heart_model.h5")
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))



--Fit model--
Epoch 1/99999999999
    462/Unknown - 4s 10ms/step - loss: 0.5699 - accuracy: 0.6797
Epoch 00001: val_loss improved from inf to 0.49901, saving model to base_heart_model.h5
462/462 [==============================] - 6s 13ms/step - loss: 0.5699 - accuracy: 0.6797 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/99999999999
456/462 [============================>.] - ETA: 0s - loss: 0.5229 - accuracy: 0.7281
Epoch 00002: val_loss improved from 0.49901 to 0.47772, saving model to base_heart_model.h5
462/462 [==============================] - 4s 8ms/step - loss: 0.5222 - accuracy: 0.7294 - val_loss: 0.4777 - val_accuracy: 0.7792
Epoch 3/99999999999
455/462 [============================>.] - ETA: 0s - loss: 0.4940 - accuracy: 0.7648
Epoch 00003: val_loss improved from 0.47772 to 0.47743, saving model to base_heart_model.h5
462/462 [==============================] - 4s 8ms/step - loss: 0.5005 - accuracy: 0.7597 - val_loss: 0.4774 - val_accuracy: 0.7706
Epoch 4/999999999